[View in Colaboratory](https://colab.research.google.com/github/karangautam/Learning-Samples/blob/master/Newyork_taxi_service.ipynb)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

gpg: keybox '/tmp/tmp_s7d3ts9/pubring.gpg' created
gpg: /tmp/tmp_s7d3ts9/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
from google.colab import auth
auth.authenticate_user()

In [35]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

KeyboardInterrupt: ignored

In [7]:
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in Drive:')
!ls drive/

# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt

Files in Drive:
Colab Notebooks  created.txt		      Getting started  Resume.docx
Court Marriage	 Daily NSE stocks report.ods  Karan


In [0]:
all_data = pd.read_csv('drive/Colab Notebooks/train.csv')
test_data = pd.read_csv('drive/Colab Notebooks/test.csv')

In [0]:
train_data= all_data.drop(['id','dropoff_datetime'], axis =1)
test_id= test_data['id']
test_data = test_data.drop(['id'],axis=1)

In [0]:
test_id = pd.DataFrame(test_id)
test_id.columns=['id']

In [0]:
import datetime

In [0]:
train_data['date'] = pd.to_datetime(train_data['pickup_datetime'])
test_data['date'] = pd.to_datetime(test_data['pickup_datetime'])

In [0]:
train_data['day']=pd.DatetimeIndex(train_data['date']).day
train_data['weekday']=pd.DatetimeIndex(train_data['date']).weekday
train_data['day']=pd.DatetimeIndex(train_data['date']).week
train_data['month']=pd.DatetimeIndex(train_data['date']).month
train_data['year']=pd.DatetimeIndex(train_data['date']).year
train_data['hour']=pd.DatetimeIndex(train_data['date']).hour
train_data['minute']=pd.DatetimeIndex(train_data['date']).minute
train_data['second']=pd.DatetimeIndex(train_data['date']).second

test_data['day']=pd.DatetimeIndex(test_data['date']).day
test_data['weekday']=pd.DatetimeIndex(test_data['date']).weekday
test_data['day']=pd.DatetimeIndex(test_data['date']).week
test_data['month']=pd.DatetimeIndex(test_data['date']).month
test_data['year']=pd.DatetimeIndex(test_data['date']).year
test_data['hour']=pd.DatetimeIndex(test_data['date']).hour
test_data['minute']=pd.DatetimeIndex(test_data['date']).minute
test_data['second']=pd.DatetimeIndex(test_data['date']).second


In [0]:
from math import radians, cos, sin, asin, sqrt

def haversine(row):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1 = row['pickup_longitude']
    lat1 = row['pickup_latitude']
    lon2 = row['dropoff_longitude']
    lat2 = row['dropoff_latitude']

    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km



In [0]:
train_data['distance'] = train_data.apply(haversine, axis=1)
test_data['distance'] = test_data.apply(haversine, axis=1)


In [16]:
X_data = train_data.drop(['pickup_datetime','pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','date','trip_duration','year'],axis=1)
X_test = test_data.drop(['pickup_datetime','pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','date','year'],axis=1)
Y_data = train_data['trip_duration']

0    1.497580
1    1.804374
2    6.381090
3    1.484566
4    1.187842
Name: distance, dtype: float64

In [0]:
X_data['vendor_id'] = X_data['vendor_id'].astype('object')
X_data['day'] = X_data['day'].astype('object')
X_data['weekday']=X_data['weekday'].astype('object')
X_data['month'] = X_data['month'].astype('object')
X_data['hour'] = X_data['hour'].astype('object')
X_data['passenger_count']=X_data['passenger_count'].astype('object')

X_test['vendor_id'] = X_test['vendor_id'].astype('object')
X_test['day'] = X_test['day'].astype('object')
X_test['weekday']=X_test['weekday'].astype('object')
X_test['month'] = X_test['month'].astype('object')
X_test['hour'] = X_test['hour'].astype('object')
X_test['passenger_count']=X_test['passenger_count'].astype('object')

In [0]:
X_data = pd.get_dummies(X_data)
X_test = pd.get_dummies(X_test)

In [0]:
X_data = X_data.convert_objects(convert_numeric=True)
X_test = X_test.convert_objects(convert_numeric=True)

In [20]:
from sklearn.preprocessing import MinMaxScaler
#train_target=pd.DataFrame(train_target.reshape(-1,1))
X_min_max = MinMaxScaler()
Y_min_max = MinMaxScaler()
train = pd.DataFrame(X_min_max.fit_transform(X_data), columns = X_data.columns)
test = pd.DataFrame(X_min_max.fit_transform(X_test), columns = X_test.columns)
test['passenger_count_8'] = 0
test['passenger_count_7'] = 0
#Y_train = pd.DataFrame(Y_min_max.fit_transform(train_target))

,vendor_id,passenger_count,day,weekday,month,year,hour,minute,second,distance,store_and_fwd_flag_N,store_and_fwd_flag_Y
0,2,1,11,0,3,2016,17,24,55,1.497580,1,0
1,1,1,23,6,6,2016,0,43,35,1.804374,1,0
2,2,1,3,1,1,2016,11,35,24,6.381090,1,0
3,2,1,14,2,4,2016,19,32,31,1.484566,1,0
4,2,1,12,5,3,2016,13,30,55,1.187842,1,0


In [21]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(Dense(256,activation='relu',input_dim=one_hot_encoded_training_predictors.shape[1]))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(1))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               3328      
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 200,961
Trainable params: 200,961
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='mse',optimizer='adam',metrics=['msle'])

In [30]:
model.fit(train,Y_data,epochs=10,verbose=2,validation_split=0.30)

Train on 1021050 samples, validate on 437594 samples
Epoch 1/10
 - 243s - loss: 34604971.5185 - mean_squared_logarithmic_error: 0.4211 - val_loss: 9717700.0059 - val_mean_squared_logarithmic_error: 0.3802
Epoch 2/10
 - 239s - loss: 34598269.7581 - mean_squared_logarithmic_error: 0.4085 - val_loss: 9742258.7736 - val_mean_squared_logarithmic_error: 0.3057
Epoch 3/10
 - 243s - loss: 34600052.1592 - mean_squared_logarithmic_error: 0.4168 - val_loss: 9740373.9445 - val_mean_squared_logarithmic_error: 0.3290
Epoch 4/10
 - 242s - loss: 34598835.2956 - mean_squared_logarithmic_error: 0.4072 - val_loss: 9715851.3226 - val_mean_squared_logarithmic_error: 0.3994
Epoch 5/10
 - 241s - loss: 34597715.8002 - mean_squared_logarithmic_error: 0.4137 - val_loss: 9721784.1492 - val_mean_squared_logarithmic_error: 0.3667
Epoch 6/10
 - 240s - loss: 34600046.6294 - mean_squared_logarithmic_error: 0.4098 - val_loss: 9716413.9123 - val_mean_squared_logarithmic_error: 0.3706
Epoch 7/10
 - 236s - loss: 34598734

In [0]:
prediction=model.predict(test)

In [0]:
v_df = pd.DataFrame(prediction)
v_df.columns=['trip_duration']

In [0]:
test_id['trip_duration'] = v_df['trip_duration']

In [0]:
test_id.to_csv('drive/submission.csv',index=False)